In [1]:

import scanpy as sc
import matplotlib.pyplot as plt
import sys
import os
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import networkx as nx
import torch
import sklearn.preprocessing
import episcanpy as epi
import sys
import os
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
# import networkx as nx # No longer needed for training script itself
import torch


adata_ATAC=sc.read("/data/cabins/sccancer/hhy/TI/adult_human_kidney_ATAC.h5ad")
adata_RNA= sc.read("/data/cabins/sccancer/hhy/TI/adult_human_kidney_RNA.h5ad")

In [2]:
adata_ATAC_copy = adata_ATAC.copy()
adata_RNA_copy = adata_RNA.copy()

In [3]:
adata_ATAC = adata_ATAC_copy 
adata_RNA = adata_RNA_copy

In [4]:
adata_ATAC.obs["cell_type"].value_counts()

cell_type
kidney loop of Henle thick ascending limb epithelial cell    7775
kidney proximal convoluted tubule epithelial cell            6384
kidney proximal straight tubule epithelial cell              4146
kidney distal convoluted tubule epithelial cell              2781
renal principal cell                                         1291
kidney connecting tubule epithelial cell                      884
kidney capillary endothelial cell                             759
epithelial cell of proximal tubule                            690
renal beta-intercalated cell                                  620
renal alpha-intercalated cell                                 613
parietal epithelial cell                                      402
connective tissue cell                                        363
leukocyte                                                     193
podocyte                                                      133
Name: count, dtype: int64

In [5]:
adata_RNA.obs["cell_type"].value_counts()

cell_type
epithelial cell of proximal tubule                           5485
kidney loop of Henle thick ascending limb epithelial cell    4435
kidney distal convoluted tubule epithelial cell              3250
kidney connecting tubule epithelial cell                     1805
renal alpha-intercalated cell                                1107
renal principal cell                                         1022
kidney capillary endothelial cell                            1008
parietal epithelial cell                                      552
podocyte                                                      463
renal beta-intercalated cell                                  349
mesangial cell                                                239
fibroblast                                                    207
leukocyte                                                      63
Name: count, dtype: int64

In [6]:
adata_ATAC = adata_ATAC[adata_ATAC.obs["cell_type"] == 'renal alpha-intercalated cell',:]
samples_to_keep = ['50-year-old stage', '52-year-old stage', '54-year-old stage', '61-year-old stage','62-year-old stage']
mask = adata_ATAC.obs['development_stage'].isin(samples_to_keep)
adata_ATAC = adata_ATAC[mask].copy()

In [7]:
adata_ATAC.obs['development_stage'].value_counts()

development_stage
61-year-old stage    182
52-year-old stage    160
62-year-old stage    121
54-year-old stage    106
50-year-old stage     44
Name: count, dtype: int64

In [8]:
adata_RNA = adata_RNA[adata_RNA.obs["cell_type"] == 'renal alpha-intercalated cell',:]
samples_to_keep = ['50-year-old stage', '52-year-old stage', '54-year-old stage', '61-year-old stage','62-year-old stage']
mask = adata_RNA.obs['development_stage'].isin(samples_to_keep)
adata_RNA = adata_RNA[mask].copy()

In [9]:
replacement_map = {
    '50-year-old stage': '50',
    '52-year-old stage': '52',
    '54-year-old stage': '54',
    '61-year-old stage': '61',
    '62-year-old stage': '62',   
}
adata_ATAC.obs['age'] = adata_ATAC.obs['development_stage'].replace(replacement_map)
adata_RNA.obs['age'] = adata_RNA.obs['development_stage'].replace(replacement_map)

In [10]:
sc.pp.filter_cells(adata_RNA, min_genes=100)
sc.pp.filter_genes(adata_RNA, min_cells=3)
# Saving count data
adata_RNA.layers["counts"] = adata_RNA.X.copy()
# Normalizing to median total counts
sc.pp.normalize_total(adata_RNA)
# Logarithmize the data
sc.pp.log1p(adata_RNA)
sc.pp.highly_variable_genes(adata_RNA, n_top_genes=2000)
adata_RNA = adata_RNA[:,adata_RNA.var['highly_variable'] ==True]
fpeak=0.03
epi.pp.binarize(adata_ATAC)
epi.pp.filter_features(adata_ATAC, min_cells=np.ceil(adata_ATAC.shape[0])*fpeak)

In [11]:
# -*- coding: utf-8 -*-
"""
Training script for CGLUE-SOE-OT model using real (preprocessed)
scRNA-seq and scATAC-seq data with ordered age labels.

This script assumes the modified CGLUE-SOE-OT model files
(e.g., cglue_soe_ot.py, cglue_soe_components_ot.py) are
accessible within the scglue package structure or Python path.
"""

import sys
import os
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
# import networkx as nx # No longer needed for training script itself
import torch

# --- Add project root to path if necessary ---
# Adjust this path if your script is located elsewhere relative to the scglue package
# project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# if project_root not in sys.path:
#     sys.path.insert(0, project_root)

try:
    # Import the modified model and config function
    import scglue
    from scglue.models.cglue_soe_ot import CGLUESOE_OT_Model, configure_dataset_cglue_soe
    # AnnDatasetWithLabels is still needed, GraphDataset is not
    from scglue.models.data import AnnDatasetWithLabels
    # from scglue.utils import config as scglue_config # Optional config import
except ImportError as e:
    print(f"Error importing modified scglue components: {e}")
    print("Please ensure the modified scglue package (with CGLUE-SOE-OT) is installed or accessible.")
    sys.exit(1)
except AttributeError as e:
    print(f"Error importing specific components (check file/class names): {e}")
    print("Ensure 'CGLUESOE_OT_Model' and 'configure_dataset_cglue_soe' exist in 'scglue.models.cglue_soe_ot'.")
    sys.exit(1)


# --- Configuration ---
SEED = 0
torch.manual_seed(SEED)
np.random.seed(SEED)
SAVE_DIR = "cglue_soe_ot_adult_human_kidney_renal_alpha_intercalated_output" # Changed save directory name
os.makedirs(SAVE_DIR, exist_ok=True)

In [12]:
# --- 1. Load Your Real Data ---
print("Loading real data...")

atac_adata= adata_ATAC
rna_adata = adata_RNA
adatas = {"rna": rna_adata, "atac": atac_adata}

# --- 2. Preprocessing Steps ---
print("Preprocessing data...")

# ** Crucial: Ensure 'age' column is ordered categorical **
# Define the correct order
age_order = ['50', '52', '54','61','62']
label_col = 'age' # MODIFY AS NEEDED
try:
    for k in adatas:
        if 'age' not in adatas[k].obs:
            raise ValueError(f"'age' column not found in adata '{k}'.")
        adatas[k].obs['age'] = pd.Categorical(
            adatas[k].obs['age'], categories=age_order, ordered=True
        )
        # Check if any values became NaN after conversion (means original values didn't match categories)
        if adatas[k].obs['age'].isnull().any():
             print(f"Warning: Some 'age' values in adata '{k}' did not match the expected categories {age_order} and resulted in NaNs.")
             # Optional: Handle NaNs, e.g., remove cells or raise error
             # adatas[k] = adatas[k][adatas[k].obs['age'].notna()].copy()
except Exception as e:
    print(f"Error processing 'age' column: {e}")
    sys.exit(1)


Loading real data...
Preprocessing data...


In [13]:
atac_adata.var['highly_variable'] = True

In [14]:

# --- 3. Define Vertices (No Graph Construction Needed) ---
# The model still needs the list of all unique features across datasets
# for the feature embedding layer.
print("Defining feature vertices...")
vertices = pd.Index(rna_adata.var_names.tolist() + atac_adata.var_names.tolist()).unique()
print(f"Total unique features (vertices): {len(vertices)}")


Defining feature vertices...
Total unique features (vertices): 17712


In [15]:

# --- 4. Configure Datasets for CGLUE-SOE-OT ---
print("Configuring datasets for CGLUE-SOE-OT...")
# ** Choose appropriate prob_model based on your preprocessing **
# 'Normal' or 'ZILN' for log-normalized RNA.
# 'Normal', 'ZILN', or 'Bernoulli' for ATAC depending on preprocessing.
try:
    configure_dataset_cglue_soe( # Use the same config function name
        adatas["rna"],
        prob_model="Normal", # Adjust if needed (e.g., "NB" if using raw counts)
        use_highly_variable=True, # Use selected HVGs
        use_label=label_col, # Use the ordered label column
        use_layer=None, # Assuming preprocessed data is in .X
        use_rep=None,   # Or specify a representation like 'X_pca' if desired
        use_obs_names=False # Data is unpaired
    )
    configure_dataset_cglue_soe( # Use the same config function name
        adatas["atac"],
        prob_model="Bernoulli", # Adjust based on ATAC preprocessing
        use_highly_variable=True, # Use selected HVGs
        use_label=label_col, # Use the ordered label column
        use_layer=None, # Assuming preprocessed data is in .X
        use_rep=None,   # Or specify a representation like 'X_lsi' if desired
        use_obs_names=False # Data is unpaired
    )
    print("Datasets configured.")
except Exception as e:
    print(f"Error configuring datasets: {e}")
    sys.exit(1)


Configuring datasets for CGLUE-SOE-OT...
[INFO] CGLUESOE_OT_Model: Dataset configured for CGLUE-SOE-OT using label 'age'.
[INFO] CGLUESOE_OT_Model: Dataset configured for CGLUE-SOE-OT using label 'age'.
Datasets configured.


In [16]:


# --- 5. Initialize CGLUE-SOE-OT Model ---
print("Initializing CGLUE-SOE-OT model...")
try:
    model = CGLUESOE_OT_Model(
        adatas=adatas,
        x2u_h_depth=2, 
        x2u_h_dim=256,                 
        u2x_h_depth=2, 
        u2x_h_dim=256, 
        dropout=0.2,
        vertices=vertices.tolist(), # Pass the combined feature list
        latent_dim=50, # Example latent dimension, adjust as needed
        random_seed=SEED
    )
    print("Model initialized.")
except Exception as e:
    print(f"Error initializing model: {e}")
    sys.exit(1)

# --- 6. Compile Model ---
print("Compiling model...")
Times = '3'
lam_kl=0.05           # Weight for KL divergence regularization 调低有利于聚类更紧密
lam_data=0.5         # Weight for data reconstruction loss 调高有利于聚类更紧密
lam_triplet=500      # Weight for triplet loss (NEEDS TUNING)
lam_ot=5.0           # Weight for Optimal Transport loss (NEEDS TUNING)
triplet_margin=3   # Margin for triplet loss (NEEDS TUNING)有可能数据本身就是按照时间序列排序的
ot_epsilon=0.1       # OT Sinkhorn regularization (NEEDS TUNING) 较小的值 (如当前的 0.1) 匹配更精确，但也可能对噪声更敏感
ot_tau=1.0          # OT Sinkhorn unbalancedness (NEEDS TUNING)
ot_max_iter=100

max_epochs=500
# ** Tune hyperparameters, especially loss weights **
try:
    model.compile(
        lr=1e-3,              # Learning rate for the single optimizer
        # lam_graph removed
        # lam_align removed
        # dsc_lr removed
        # train_ratio removed
        lam_kl=lam_kl,           # Weight for KL divergence regularization 调低有利于聚类更紧密
        lam_data=lam_data,         # Weight for data reconstruction loss 调高有利于聚类更紧密
        lam_triplet=lam_triplet,      # Weight for triplet loss (NEEDS TUNING)
        lam_ot=lam_ot,           # Weight for Optimal Transport loss (NEEDS TUNING)
        triplet_margin=triplet_margin,   # Margin for triplet loss (NEEDS TUNING)有可能数据本身就是按照时间序列排序的
        ot_epsilon=ot_epsilon,       # OT Sinkhorn regularization (NEEDS TUNING) 较小的值 (如当前的 0.1) 匹配更精确，但也可能对噪声更敏感
        ot_tau=ot_tau,           # OT Sinkhorn unbalancedness (NEEDS TUNING)
        ot_max_iter=ot_max_iter,      # OT Sinkhorn iterations
         #weight_decay=1e-5   # Example: Add optimizer args directly
    )
    print("Model compiled.")
except Exception as e:
    print(f"Error compiling model: {e}")
    sys.exit(1)

# --- 7. Train Model ---
print("Fitting model...")
# Fit the model (No GraphDataset needed)
# Adjust training parameters as needed
try:
    model.fit(
        adatas=adatas,            # Pass adatas dictionary
        # graph removed
        val_split=0.2,            # Use 10% of data for validation
        data_batch_size=256,      # Adjust based on memory
        # graph_batch_size removed
        max_epochs=max_epochs,           # Set a reasonable number of epochs
        patience=35,              # Early stopping patience
        reduce_lr_patience=20,    # LR scheduling patience
        wait_n_lrs=3,             # Wait for LR reductions before stopping
        num_workers=0,            # Set > 0 for parallel data loading if needed
        directory=SAVE_DIR        # Directory for logs and checkpoints
    )
    print("Model fitting finished.")
except Exception as e:
    print(f"An error occurred during model fitting: {e}")
    import traceback
    traceback.print_exc()
    sys.exit(1)




Initializing CGLUE-SOE-OT model...
[INFO] autodevice: Using GPU 3 as computation device.
Model initialized.
Compiling model...
Model compiled.
Fitting model...



For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.



[INFO] CGLUESOE_OT_Trainer: [Epoch 10] Train={'x_rna_nll': '1126.4853', 'x_rna_kl': '19.8496', 'x_rna_elbo': '1127.4777', 'x_atac_nll': '14097.1682', 'x_atac_kl': '19.9045', 'x_atac_elbo': '14098.1635', 'triplet_loss': '0.0435', 'ot_loss': '50.9759', 'total_loss': '7889.4627'}, Val={'x_rna_nll': '1117.3446', 'x_rna_kl': '10.7171', 'x_rna_elbo': '1117.8804', 'x_atac_nll': '13801.8008', 'x_atac_kl': '9.9981', 'x_atac_elbo': '13802.3008', 'triplet_loss': '1.8542', 'ot_loss': '15.5868', 'total_loss': '8465.1177'}
[INFO] CGLUESOE_OT_Trainer: [Epoch 20] Train={'x_rna_nll': '1057.0722', 'x_rna_kl': '16.8599', 'x_rna_elbo': '1057.9152', 'x_atac_nll': '12285.4928', 'x_atac_kl': '22.7414', 'x_atac_elbo': '12286.6299', 'triplet_loss': '0.0388', 'ot_loss': '43.0661', 'total_loss': '6907.0094'}, Val={'x_rna_nll': '1044.4372', 'x_rna_kl': '12.4781', 'x_rna_elbo': '1045.0611', 'x_atac_nll': '12066.6567', 'x_atac_kl': '15.6403', 'x_atac_elbo': '12067.4390', 'triplet_loss': '1.6916', 'ot_loss': '20.331

[INFO] CGLUESOE_OT_Trainer: [Epoch 180] Train={'x_rna_nll': '-100.6524', 'x_rna_kl': '132.3646', 'x_rna_elbo': '-94.0342', 'x_atac_nll': '7940.5983', 'x_atac_kl': '171.3227', 'x_atac_elbo': '7949.1645', 'triplet_loss': '0.0229', 'ot_loss': '30.7774', 'total_loss': '4092.9147'}, Val={'x_rna_nll': '-114.2574', 'x_rna_kl': '127.2886', 'x_rna_elbo': '-107.8930', 'x_atac_nll': '8782.0293', 'x_atac_kl': '158.5486', 'x_atac_elbo': '8789.9570', 'triplet_loss': '2.1633', 'ot_loss': '8.8293', 'total_loss': '5466.8054'}
[INFO] CGLUESOE_OT_Trainer: [Epoch 190] Train={'x_rna_nll': '-168.1045', 'x_rna_kl': '135.8002', 'x_rna_elbo': '-161.3145', 'x_atac_nll': '7875.1059', 'x_atac_kl': '176.6118', 'x_atac_elbo': '7883.9363', 'triplet_loss': '0.0086', 'ot_loss': '30.9161', 'total_loss': '4020.1999'}, Val={'x_rna_nll': '-183.3658', 'x_rna_kl': '130.4768', 'x_rna_elbo': '-176.8419', 'x_atac_nll': '8730.7529', 'x_atac_kl': '158.8854', 'x_atac_elbo': '8738.6973', 'triplet_loss': '2.1053', 'ot_loss': '8.456

[INFO] CGLUESOE_OT_Trainer: [Epoch 340] Train={'x_rna_nll': '-1050.9519', 'x_rna_kl': '170.4029', 'x_rna_elbo': '-1042.4318', 'x_atac_nll': '7294.6477', 'x_atac_kl': '209.9863', 'x_atac_elbo': '7305.1470', 'triplet_loss': '0.0180', 'ot_loss': '18.7553', 'total_loss': '3234.1143'}, Val={'x_rna_nll': '-1079.1389', 'x_rna_kl': '164.0115', 'x_rna_elbo': '-1070.9384', 'x_atac_nll': '8497.4526', 'x_atac_kl': '196.9851', 'x_atac_elbo': '8507.3018', 'triplet_loss': '1.4974', 'ot_loss': '8.7946', 'total_loss': '4510.8618'}
[INFO] CGLUESOE_OT_Trainer: [Epoch 350] Train={'x_rna_nll': '-1101.1122', 'x_rna_kl': '170.5562', 'x_rna_elbo': '-1092.5844', 'x_atac_nll': '7310.8049', 'x_atac_kl': '211.4778', 'x_atac_elbo': '7321.3789', 'triplet_loss': '0.0251', 'ot_loss': '17.5833', 'total_loss': '3214.8647'}, Val={'x_rna_nll': '-1126.3674', 'x_rna_kl': '164.8452', 'x_rna_elbo': '-1118.1251', 'x_atac_nll': '8506.8315', 'x_atac_kl': '196.3761', 'x_atac_elbo': '8516.6504', 'triplet_loss': '1.2401', 'ot_loss

[INFO] CGLUESOE_OT_Trainer: [Epoch 500] Train={'x_rna_nll': '-1512.3905', 'x_rna_kl': '190.2935', 'x_rna_elbo': '-1502.8759', 'x_atac_nll': '7178.8120', 'x_atac_kl': '223.7767', 'x_atac_elbo': '7190.0008', 'triplet_loss': '0.0043', 'ot_loss': '9.9574', 'total_loss': '2895.5134'}, Val={'x_rna_nll': '-1537.9074', 'x_rna_kl': '182.3521', 'x_rna_elbo': '-1528.7898', 'x_atac_nll': '8545.1743', 'x_atac_kl': '208.2588', 'x_atac_elbo': '8555.5874', 'triplet_loss': '1.2653', 'ot_loss': '9.9801', 'total_loss': '4195.9703'}
Model fitting finished.


In [17]:
# --- 8. Post-Training Analysis ---
print("Extracting embeddings...")
try:
    # Use a different key for the new model's embeddings
    embedding_key = "X_cgluesoe_ot"
    rna_adata.obsm[embedding_key] = model.encode_data("rna", rna_adata)
    atac_adata.obsm[embedding_key] = model.encode_data("atac", atac_adata)

    # Get feature embeddings from the nn.Embedding layer
    feature_embeddings = model.get_feature_embeddings()
    feature_embeddings_df = pd.DataFrame(feature_embeddings, index=model.vertices)

    print("RNA embeddings shape:", rna_adata.obsm[embedding_key].shape)
    print("ATAC embeddings shape:", atac_adata.obsm[embedding_key].shape)
    print("Feature embeddings shape:", feature_embeddings_df.shape)

    # --- Optional: Save model and embeddings ---
    final_model_path = os.path.join(SAVE_DIR, "cglue_soe_ot_final_model.dill")
    print(f"Saving final model to {final_model_path}")
    model.save(final_model_path)

    rna_save_path = os.path.join(SAVE_DIR, "rna_adata_with_embeddings.h5ad")
    atac_save_path = os.path.join(SAVE_DIR, "atac_adata_with_embeddings.h5ad")
    feat_save_path = os.path.join(SAVE_DIR, "feature_embeddings.csv")

    # print(f"Saving RNA AnnData to {rna_save_path}")
    # rna_adata.write_h5ad(rna_save_path)
    # print(f"Saving ATAC AnnData to {atac_save_path}")
    # atac_adata.write_h5ad(atac_save_path)
    # print(f"Saving feature embeddings to {feat_save_path}")
    # feature_embeddings_df.to_csv(feat_save_path)


    # --- Optional: Visualization (UMAP) ---
    print("Generating UMAP visualization...")
    # Combine embeddings and obs for visualization
    all_embeddings = np.concatenate([rna_adata.obsm[embedding_key], atac_adata.obsm[embedding_key]], axis=0)
    all_obs = pd.concat([
        rna_adata.obs[[label_col]].assign(modality='rna'), # Use label_col
        atac_adata.obs[[label_col]].assign(modality='atac') # Use label_col
    ], axis=0)
    # Create combined obs_names for the visualization AnnData
    vis_obs_names = rna_adata.obs_names.tolist() + atac_adata.obs_names.tolist()
    if len(vis_obs_names) != len(all_obs): # Basic check
         print(f"Warning: Length mismatch between obs_names ({len(vis_obs_names)}) and combined obs ({len(all_obs)}). Using default index.")
         vis_obs_names = pd.RangeIndex(start=0, stop=len(all_obs), step=1)


    vis_adata = ad.AnnData(all_embeddings, dtype=all_embeddings.dtype) # Ensure dtype is set
    vis_adata.obs = all_obs
    vis_adata.obs_names = vis_obs_names # Assign combined names
    vis_adata.obsm['X_cgluesoe_ot'] = all_embeddings # Store for scanpy functions

    print("Running neighbors and UMAP...")
    sc.pp.neighbors(vis_adata, use_rep='X_cgluesoe_ot', n_neighbors=15, random_state=SEED)
    sc.tl.umap(vis_adata, min_dist=0.3, random_state=SEED) # Adjust min_dist for spread

    print("Plotting UMAP...")
    umap_save_path = f'_cglue_soe_ot_adult_human_kidney_renal_alpha_intercalated_results_{Times}.png' # Updated filename
    sc.pl.umap(
        vis_adata,
        color=['modality', label_col], # Color by label_col
        title=['UMAP by Modality', f'UMAP by {label_col.capitalize()}'],
        save=umap_save_path,
        show=False # Set to True to display interactively if possible
    )
    print(f"UMAP plots saved to figures/ directory ({umap_save_path}).")

except Exception as e:
    print(f"An error occurred during post-training analysis: {e}")
    import traceback
    traceback.print_exc()

print("Script finished.")

Extracting embeddings...
RNA embeddings shape: (1107, 50)
ATAC embeddings shape: (613, 50)
Feature embeddings shape: (17712, 50)
Saving final model to cglue_soe_ot_adult_human_kidney_renal_alpha_intercalated_output/cglue_soe_ot_final_model.dill
[INFO] CGLUESOE_OT_Model: Model state saved to cglue_soe_ot_adult_human_kidney_renal_alpha_intercalated_output/cglue_soe_ot_final_model.dill
Generating UMAP visualization...
Running neighbors and UMAP...


Plotting UMAP...
UMAP plots saved to figures/ directory (_cglue_soe_ot_adult_human_kidney_renal_alpha_intercalated_results_3.png).
Script finished.


In [18]:
print("Plotting loss curves...")
try:
    # Define where to save the plot
    loss_plot_path = os.path.join(SAVE_DIR, f"cglue_soe_ot_loss_curves_adult_human_kidney_renal_alpha_intercalated_{Times}.png")
    # Call the plotting method
    model.plot_loss_curves(save_path=loss_plot_path)
    print(f"Loss curves saved to {loss_plot_path}")
except Exception as e:
    print(f"An error occurred during loss plotting: {e}")
    import traceback
    traceback.print_exc()


Plotting loss curves...
[INFO] CGLUESOE_OT_Model: Loss curves saved to cglue_soe_ot_adult_human_kidney_renal_alpha_intercalated_output/cglue_soe_ot_loss_curves_adult_human_kidney_renal_alpha_intercalated_3.png
Loss curves saved to cglue_soe_ot_adult_human_kidney_renal_alpha_intercalated_output/cglue_soe_ot_loss_curves_adult_human_kidney_renal_alpha_intercalated_3.png


In [19]:
import csv
import pickle
# 定义你的超参数 (与上面相同)
hyperparameters = {
    "lam_kl": lam_kl,
    "lam_data": lam_data,
    "lam_triplet": lam_triplet,
    "lam_ot": lam_ot,
    "triplet_margin": triplet_margin,
    "ot_epsilon": ot_epsilon,
    "ot_tau": ot_tau,
    "ot_max_iter": ot_max_iter,
    "max_epochs": max_epochs
}

# 定义要保存的PKL文件名
csv_filename = os.path.join(SAVE_DIR, "hyperparameters_renal_alpha_intercalated.csv")

# 写入CSV文件
try:
    with open(csv_filename, 'a', newline='') as csvfile:
        # 定义CSV写入器
        writer = csv.writer(csvfile)

        # 写入表头 (可选，但推荐)
        writer.writerow([f'ParameterName_{Times}', 'Value'])

        # 遍历字典并写入每一行
        for name, value in hyperparameters.items():
            writer.writerow([name, value])
    print(f"超参数已成功保存到 {csv_filename}")
except IOError:
    print(f"错误：无法写入文件 {csv_filename}")

超参数已成功保存到 cglue_soe_ot_adult_human_kidney_renal_alpha_intercalated_output/hyperparameters_renal_alpha_intercalated.csv
